# Passo-a-passo

In [ ]:
# Importar bibliotecas utilizadas
import pandas as pd

import numpy as np

import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer

# Regular expression
import re

from wordcloud import WordCloud

import matplotlib.pyplot as plt

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from nltk import ngrams

In [ ]:
# Set of predefined words NLTK (Natural Language Toolkit)
import nltk 

#nltk.download('all')

from nltk.corpus import stopwords 

set(stopwords.words('english'))

from nltk.tokenize import word_tokenize 


#from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.stem import PorterStemmer

porter = PorterStemmer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()


#print(stopwords.words('english'))

In [ ]:
# Coleta dados
catalogoVulnerabilidades_bruto = pd.read_csv('https://www.cisa.gov/sites/default/files/csv/known_exploited_vulnerabilities.csv', header=0, sep=',')
#catalogoVulnerabilidades = pd.read_csv('dados/known_exploited_vulnerabilities.csv', header=0, sep=',')

# Mantém o DataFrame obtido originalmente sem alterações
catalogoVulnerabilidades = catalogoVulnerabilidades_bruto.copy()

# Criar coluna booleana state com 0 para Unknown e 1 para Known
catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'] = np.where(catalogoVulnerabilidades['knownRansomwareCampaignUse'] == 'Known', 1, 0 )

#Reordenando as colunas
catalogoVulnerabilidades = catalogoVulnerabilidades[['cveID',
                                                     'vendorProject',
                                                     'product',
                                                     'vulnerabilityName',
                                                     'dateAdded',
                                                     'shortDescription',
                                                     'requiredAction',
                                                     'dueDate',
                                                     'knownRansomwareCampaignUse',
                                                     'codigoKnownRansomwareCampaignUse',
                                                     'notes']]    



In [ ]:
# Verificar tipos de colunas
catalogoVulnerabilidades.info()

In [ ]:
# Excluir a coluna notes
catalogoVulnerabilidades.drop(columns=['cveID', 'notes'], inplace = True)

In [ ]:
# Verificar alguns dados
display(catalogoVulnerabilidades)

In [ ]:
# Transformar tipos de dados para datas
catalogoVulnerabilidades.dateAdded = pd.to_datetime(catalogoVulnerabilidades.dateAdded)
catalogoVulnerabilidades.dueDate = pd.to_datetime(catalogoVulnerabilidades.dueDate)

##Tranforma a coluna codigoKnownRansomwareCampaignUse para boolean
catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'] = catalogoVulnerabilidades['codigoKnownRansomwareCampaignUse'].astype('bool')

##Transforma colunas em string
catalogoVulnerabilidades['shortDescription'] = catalogoVulnerabilidades['shortDescription'].astype('string')

In [ ]:
# Verificar tipos de colunas após transformação
catalogoVulnerabilidades.info()

In [ ]:
# Função para tokenizar as palavras de uma coluna de um DataFrame
def tokeniza_palavras_coluna (dataframe, coluna):
    palavras = ' '.join([word for word in dataframe[coluna]])    
    # Realiza a tokenização
    tokens = word_tokenize(palavras)
    return tokens

In [ ]:
# Função para preparar exibição de gráfico de ocorrências de palavras em uma coluna de um DataFrame
def prepara_para_grafico (dataframe, coluna):
    # Realiza a tokenização
    tokens = tokeniza_palavras_coluna(dataframe, coluna)
    # Obtém a frequência de ocorrências do token
    frequencia = nltk.FreqDist(tokens)
    pd_frequencia = pd.DataFrame({"token": list(frequencia.keys()),
                                 "frequencia": list(frequencia.values())})
    return pd_frequencia


In [ ]:
# DataFrame com pontos e caracteres especiais
pd_frequencia_com_pontos = prepara_para_grafico(catalogoVulnerabilidades, 'shortDescription')

pd_frequencia_com_pontos.nlargest(columns = "frequencia", n = 10)

In [ ]:
# Função para exibir gráfico de Pareto de ocorrências de palavras em uma coluna de um DataFrame
def exibe_pareto (dados, coluna, quantidade_colunas, cor, titulo):
    plt.figure(figsize=(10, 6))
    x = sns.barplot(data = dados.nlargest(columns = coluna, n = quantidade_colunas), x = "token", y = "frequencia", color = cor)
    x.set(ylabel = "Quantidade", xlabel = "Tokens", title = titulo)
    plt.show()

In [ ]:
# Exibe gráfico de pareto contendo caracteres especiais e stop words
exibe_pareto(pd_frequencia_com_pontos, 'frequencia', 10, 'purple', 'Contendo caracteres especiais e stop words')

In [ ]:
# Função para limpar os textos das colunas do dataframe
def limpa_coluna(corpus):
    return re.sub(r'[^a-zA-Z0-9\s]', '', corpus)

In [ ]:
# Limpa a coluna 
pd_catalogoVulnerabilidades_coluna_limpa = pd.DataFrame(catalogoVulnerabilidades)
pd_catalogoVulnerabilidades_coluna_limpa['shortDescription'] = catalogoVulnerabilidades['shortDescription'].apply(lambda x: limpa_coluna(str(x)))
pd_catalogoVulnerabilidades_coluna_limpa.shortDescription

In [ ]:
# Tokeniza DataFrame limpo
tokens_coluna_limpa = tokeniza_palavras_coluna(pd_catalogoVulnerabilidades_coluna_limpa, 'shortDescription')

In [ ]:
# Função para remover stop words
def remove_stop_words(word_list, stop_word_list):
    lista_sem_stop_words = [word for word in word_list 
                            if word.lower() not in stop_word_list]
    return lista_sem_stop_words

In [ ]:
# Carrega lista de stop words do inglês e acrescenta algumas palavras
sw_vulnerabilidade_ciberneticas = ["could", "vulnerability"] # Ver quais stop words adicionar
sw_en = list(set(stopwords.words('english')))
sw_en_plus = sw_en + sw_vulnerabilidade_ciberneticas



In [ ]:
# Remove stop words
tokens_sem_stop_words = remove_stop_words(tokens_coluna_limpa, sw_en_plus)

In [ ]:
# Cria DataFrame sem stop words
frequencia = nltk.FreqDist(tokens_sem_stop_words)
pd_frequencia_sem_stop_words = pd.DataFrame({"token": list(frequencia.keys()),
                                 "frequencia": list(frequencia.values())})

In [ ]:
# Exibe gráfico de pareto sem caracteres especiais e sem stop words
exibe_pareto(pd_frequencia_sem_stop_words, 'frequencia', 10, 'blue', 'Sem stop words e caracteres especiais')

In [ ]:
# Função de Stemming
def stemming (texto):
    stemmings = [porter.stem(word) for word in word_tokenize(texto)]
    return ' '.join(stemmings)

In [ ]:
# Aplicando Stemming
pd_catalogoVulnerabilidades_stemming = pd_catalogoVulnerabilidades_coluna_limpa.copy()
pd_catalogoVulnerabilidades_stemming['shortDescription'] = pd_catalogoVulnerabilidades_stemming['shortDescription'].apply(lambda x: stemming(str(x)))
pd_catalogoVulnerabilidades_stemming.shortDescription

In [ ]:
# Cria DataFrame com stemming

stemming_contagem = ' '.join([word for word in pd_catalogoVulnerabilidades_stemming.shortDescription])

stemming_contagem = word_tokenize(stemming_contagem)

stemming_contagem = remove_stop_words(stemming_contagem, sw_en_plus)

stemming_frequencia = nltk.FreqDist(stemming_contagem)

df_frequencia_stemming = pd.DataFrame({"token": list(stemming_frequencia.keys()),
                                 "frequencia": list(stemming_frequencia.values())})


df_frequencia_stemming.nlargest(columns = "frequencia", n = 10)

In [ ]:
# Gráfico de Pareto depois do stemming
plt.figure(figsize=(10, 6))
x = sns.barplot(data = df_frequencia_stemming.nlargest(columns = "frequencia", n = 10), x = "token", y = "frequencia", color = 'yellow')
x.set(ylabel = "Quantidade", xlabel = "Tokens", title = 'Stemming')
plt.show()


In [ ]:
# Função de Lematização
def lematizacao (texto):
    lemas = [lemmatizer.lemmatize(word) for word in word_tokenize(texto)]
    return ' '.join(lemas)

In [ ]:
# Aplicando Lemmatization
pd_catalogoVulnerabilidades_lemmatization = pd_catalogoVulnerabilidades.copy()
pd_catalogoVulnerabilidades_lemmatization['shortDescription'] = pd_catalogoVulnerabilidades_lemmatization['shortDescription'].apply(lambda x: lematizacao(str(x)))
pd_catalogoVulnerabilidades_lemmatization.shortDescription

In [ ]:
# Cria DataFrame com Lemmatization

lemmatization_contagem = ' '.join([word for word in pd_catalogoVulnerabilidades_lemmatization.shortDescription])

lemmatization_contagem = word_tokenize(lemmatization_contagem)

lemmatization_contagem = remove_stop_words(lemmatization_contagem, sw_en_plus)

lemmatization_frequencia = nltk.FreqDist(lemmatization_contagem)

df_frequencia_lemmatization = pd.DataFrame({"token": list(lemmatization_frequencia.keys()),
                                 "frequencia": list(lemmatization_frequencia.values())})


df_frequencia_lemmatization.nlargest(columns = "frequencia", n = 10)

In [ ]:
# Gráfico de Pareto depois do Lemmatization
plt.figure(figsize=(10, 6))
x = sns.barplot(data = df_frequencia_lemmatization.nlargest(columns = "frequencia", n = 10), x = "token", y = "frequencia", color = 'green')
x.set(ylabel = "Quantidade", xlabel = "Tokens", title = 'Lemmatization')
plt.show()


In [ ]:
# Vetorizar coluna
vetorizar = CountVectorizer(lowercase = False, max_features=100)
bow = vetorizar.fit_transform(catalogoVulnerabilidades.shortDescription)

print(bow.shape)

In [ ]:
# Tranforma matriz esparsa em Dataframe
df_shortDescription_bow = pd.DataFrame.sparse.from_spmatrix(bow, columns=vetorizar.get_feature_names_out())

df_shortDescription_bow.head()

In [ ]:
# Wordcloud
shortDescription_all_words = ' '.join([word for word in pd_catalogoVulnerabilidades.shortDescription])

# Quantidade de palavras
print(len(shortDescription_all_words))

In [ ]:
# Gerar wordcloud

## https://amueller.github.io/word_cloud/generated/wordcloud.WordCloud.html

shortDescription_wc = WordCloud(width= 800, height= 500, max_font_size = 110, collocations=False).generate(shortDescription_all_words)

In [ ]:
# Plotar wordcloud
plt.figure(figsize=(10, 6))
plt.imshow(shortDescription_wc, interpolation='bilinear') #ver outras interpolações
plt.axis("off")
plt.show()

In [ ]:
# TF-IDF
tfidf = TfidfVectorizer(lowercase = False, max_features=100)
regressao_logistica = LogisticRegression(solver = "lbfgs")

catalogoVulnerabilidades_tfidf_bruto = tfidf.fit_transform(pd_catalogoVulnerabilidades['shortDescription'])

catalogoVulnerabilidades_treino, catalogoVulnerabilidades_teste, catalogoVulnerabilidades_classe_treino, catalogoVulnerabilidades_classe_teste = train_test_split(catalogoVulnerabilidades_tfidf_bruto,
                                                                                                                                                                  pd_catalogoVulnerabilidades['knownRansomwareCampaignUse'], random_state = 42)

regressao_logistica.fit(catalogoVulnerabilidades_treino, catalogoVulnerabilidades_classe_treino)

acuracia_catalogoVulnerabilidades_tfidf_bruto = regressao_logistica.score(catalogoVulnerabilidades_teste, catalogoVulnerabilidades_classe_teste)

print(acuracia_catalogoVulnerabilidades_tfidf_bruto)

In [ ]:
# Ngrams
tfidf = TfidfVectorizer(lowercase=False, ngram_range=(1,2))
catalogoVulnerabilidades_vetor_tfidf = tfidf.fit_transform(pd_catalogoVulnerabilidades['shortDescription'])

catalogoVulnerabilidades_vetor_tfidf_treino, catalogoVulnerabilidades_vetor_tfidf_teste, catalogoVulnerabilidades_vetor_tfidf_classe_treino, catalogoVulnerabilidades_vetor_tfidf_classe_teste = train_test_split(catalogoVulnerabilidades_vetor_tfidf,
                                                                                                                                                                  pd_catalogoVulnerabilidades['knownRansomwareCampaignUse'], random_state = 42)

regressao_logistica.fit(catalogoVulnerabilidades_vetor_tfidf_treino, catalogoVulnerabilidades_vetor_tfidf_classe_treino)

acuracia_catalogoVulnerabilidades_vetor_tfidf_ngrams = regressao_logistica.score(catalogoVulnerabilidades_vetor_tfidf_teste, catalogoVulnerabilidades_vetor_tfidf_classe_teste)

print(acuracia_catalogoVulnerabilidades_vetor_tfidf_ngrams)

In [ ]:
# Pesos de cada termo
pesos_termo = pd.DataFrame(
    regressao_logistica.coef_[0].T,
    index = tfidf.get_feature_names_out()
)

pesos_termo.nlargest(10, 0)